# Learning Objectives

- Illustrate how to improve the retrieval process of RAG using:
    - Query Expansion
    - Hypothetical Questions


# Setup

In [ ]:
### If there is error execute the cell twice in google collab

!pip install -U chromadb

In [ ]:
### If there is error execute the cell twice in google collab

!pip install -q tiktoken==0.9.0 \
                langchain==0.3.20 \
                langchain-chroma==0.2.2 \
                langchain-community==0.3.19 \
                sentence-transformers \
                langchain-groq
                # chromadb==0.6.3
                # langchain-openai==0.3.9
                #openai==1.66.3 \

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
import os
import chromadb

from langchain_chroma import Chroma
from langchain_groq import ChatGroq


from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

from google.colab import userdata

In [ ]:
from google.colab import userdata


groq_api_key = userdata.get('GROQ_API_KEY')

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/tmp/ipython-input-1656931266.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0, api_key = groq_api_key)

To illustrate the techniques of improving retrieval, let us set up an ephemeral Chroma database with a few documents.

In [ ]:
chromadb_client = chromadb.EphemeralClient()

In [ ]:
documents = [
    Document(
        id=1,
        page_content="We design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products.",
        metadata={"year": 2023, "section": "business"}
    ),
    Document(
        id=2,
        page_content="We have previously experienced and may in the future experience launch and production ramp delays for new products and features. For example, we encountered unanticipated supplier issues that led to delays during the initial ramp of our first Model X and experienced challenges with a supplier and with ramping full automation for certain of our initial Model 3 manufacturing processes. In addition, we may introduce in the future new or unique manufacturing processes and design features for our products. As we expand our vehicle offerings and global footprint, there is no guarantee that we will be able to successfully and timely introduce and scale such processes or features.",
        metadata={"year": 2023, "section": "risk_factors"}
    ),
    Document(
        id=3,
        page_content="We recognize the importance of assessing, identifying, and managing material risks associated with cybersecurity threats, as such term is defined in Item 106(a) of Regulation S-K. These risks include, among other things: operational risks, intellectual property theft, fraud, extortion, harm to employees or customers and violation of data privacy or security laws. Identifying and assessing cybersecurity risk is integrated into our overall risk management systems and processes. Cybersecurity risks related to our business, technical operations, privacy and compliance issues are identified and addressed through a multi-faceted approach including third party assessments, internal IT Audit, IT security, governance, risk and compliance reviews. To defend, detect and respond to cybersecurity incidents, we, among other things: conduct proactive privacy and cybersecurity reviews of systems and applications, audit applicable data policies, perform penetration testing using external third-party tools and techniques to test security controls, operate a bug bounty program to encourage proactive vulnerability reporting, conduct employee training, monitor emerging laws and regulations related to data protection and information security (including our consumer products) and implement appropriate changes.",
        metadata={"year": 2023, "section": "cyber_security"}
    ),
    Document(
        id=4,
        page_content="The automotive segment includes the design, development, manufacturing, sales and leasing of high-performance fully electric vehicles as well as sales of automotive regulatory credits. Additionally, the automotive segment also includes services and other, which includes non-warranty after- sales vehicle services and parts, sales of used vehicles, retail merchandise, paid Supercharging and vehicle insurance revenue. The energy generation and storage segment includes the design, manufacture, installation, sales and leasing of solar energy generation and energy storage products and related services and sales of solar energy systems incentives.",
        metadata={"year": 2022, "section": "business"}
    ),
    Document(
        id=5,
        page_content="Since the first quarter of 2020, there has been a worldwide impact from the COVID-19 pandemic. Government regulations and shifting social behaviors have, at times, limited or closed non-essential transportation, government functions, business activities and person-to-person interactions. Global trade conditions and consumer trends that originated during the pandemic continue to persist and may also have long-lasting adverse impact on us and our industries independently of the progress of the pandemic.",
        metadata={"year": 2022, "section": "risk_factors"}
    ),
    Document(
        id=6,
        page_content="The German Umweltbundesamt issued our subsidiary in Germany a notice and fine in the amount of 12 million euro alleging its non-compliance under applicable laws relating to market participation notifications and take-back obligations with respect to end-of-life battery products required thereunder. In response to Tesla’s objection, the German Umweltbundesamt issued Tesla a revised fine notice dated April 29, 2021 in which it reduced the original fine amount to 1.45 million euro. This is primarily relating to administrative requirements, but Tesla has continued to take back battery packs, and filed a new objection in June 2021. A hearing took place on November 24, 2022, and the parties reached a settlement which resulted in a further reduction of the fine to 600,000 euro. Both parties have waived their right to appeal.",
        metadata={"year": 2022, "section": "legal_proceedings"}
    )
]

In [ ]:
type(documents[0])

langchain_core.documents.base.Document

We can now point our vector store to the Chroma client and add these documents to the vector store as in previous modules.

In [ ]:
vectorstore = Chroma(
    collection_name="full_document_chunks",
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client
)

In [ ]:
chromadb_client.list_collections()

['full_document_chunks']

In [ ]:
vectorstore.add_documents(
    documents=documents
)

['1', '2', '3', '4', '5', '6']

In [ ]:
vectorstore


In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        'k': 3
    }
)

# Query Expansion

In query expansion, we ask the LLM to generate variations of the original user query. Then, we use each of these variations to retrieve the relevant context. The final context used is the **unique** set of documents that are retreived across all the query expansions.

In [ ]:
query_expansion_system_message = """
You are an financial domain expert assisting in answering questions related to 10-k reports.
Perform query expansion on the question below. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

If there are acronyms or words you are not familiar with, do not try to rephrase them.

Return at least 3 versions of the question as a list.
Generate only a list of questions, each question in a new line.
Do not number the list of questions or use bullet points.
Do not mention anything before or after the list.
"""

user_message_template="""
<Question>
{question}
</Question>
"""

In [ ]:
user_input = "Any specific fines levied on the company in 2022?"

In [ ]:
query_expansions = llm.invoke(
    [
        ('system', query_expansion_system_message),
        ('user', user_message_template.format(question=user_input))
    ]
)

In [ ]:
type(query_expansions.content)

str

In [ ]:
print(query_expansions)

content='What fines were imposed on the company during 2022?\nWere any penalties assessed against the company in 2022?\nDid the company receive any specific fines in 2022?\nCan you list any fines the company faced in 2022?\nWere any regulatory fines levied on the firm in 2022?' additional_kwargs={'reasoning_content': 'The user wants query expansion: produce at least 3 versions of the question, each on a new line, no numbering or bullets, no extra text. The question: "Any specific fines levied on the company in 2022?" Need to produce variations: synonyms for "specific fines", "levied", "company", "in 2022". Provide at least 3 versions.\n\nPotential variations:\n- What fines were imposed on the company during 2022?\n- Were there any penalties assessed against the company in 2022?\n- Did the company receive any specific fines in 2022?\n- Can you list any fines the company faced in 2022?\n- Were any regulatory fines levied on the firm in 2022?\n\nWe need at least 3. Provide list, each on n

In [ ]:
print(query_expansions.additional_kwargs['reasoning_content'])

The user wants query expansion: produce at least 3 versions of the question, each on a new line, no numbering or bullets, no extra text. The question: "Any specific fines levied on the company in 2022?" Need to produce variations: synonyms for "specific fines", "levied", "company", "in 2022". Provide at least 3 versions.

Potential variations:
- What fines were imposed on the company during 2022?
- Were there any penalties assessed against the company in 2022?
- Did the company receive any specific fines in 2022?
- Can you list any fines the company faced in 2022?
- Were any regulatory fines levied on the firm in 2022?

We need at least 3. Provide list, each on new line, no numbering, no bullets. Let's output 5 lines.


In [ ]:
print(query_expansions.content)

What fines were imposed on the company during 2022?
Were any penalties assessed against the company in 2022?
Did the company receive any specific fines in 2022?
Can you list any fines the company faced in 2022?
Were any regulatory fines levied on the firm in 2022?


In [ ]:
query_expansions_list = query_expansions.content.strip().split("\n")

In [ ]:
query_expansions_list

['What fines were imposed on the company during 2022?',
 'Were any penalties assessed against the company in 2022?',
 'Did the company receive any specific fines in 2022?',
 'Can you list any fines the company faced in 2022?',
 'Were any regulatory fines levied on the firm in 2022?']

In [ ]:
expanded_context_list = []

In [ ]:
for query in query_expansions_list:
    expanded_context_list.extend([d.page_content for d in retriever.invoke(query)])

In [ ]:
len(expanded_context_list)

15

In [ ]:
expanded_context_list

['The German Umweltbundesamt issued our subsidiary in Germany a notice and fine in the amount of 12 million euro alleging its non-compliance under applicable laws relating to market participation notifications and take-back obligations with respect to end-of-life battery products required thereunder. In response to Tesla’s objection, the German Umweltbundesamt issued Tesla a revised fine notice dated April 29, 2021 in which it reduced the original fine amount to 1.45 million euro. This is primarily relating to administrative requirements, but Tesla has continued to take back battery packs, and filed a new objection in June 2021. A hearing took place on November 24, 2022, and the parties reached a settlement which resulted in a further reduction of the fine to 600,000 euro. Both parties have waived their right to appeal.',
 'We recognize the importance of assessing, identifying, and managing material risks associated with cybersecurity threats, as such term is defined in Item 106(a) of 

In [ ]:
final_context_documents = set(expanded_context_list)

In [ ]:
len(final_context_documents)

5

# Hypothetical Questions

In this approach, we generate 3 hypothetical questions that can be answered with each document chunk. These hypothetical questions are then seperately indexed into the vector database (along with the parent document chunk ids as metadata). For each query, we then retrieve relevant hypothetical questions first and the then retrieve the associated chunks as the second step. Note that the retrieval is focused on the comparison between the user query and hypothetical questions.

In [ ]:
hypothetical_questions_system_message = """
Generate a list of exactly 3 hypothetical questions that the document presented in the input could be used to answer.
Generate only a list of questions, each question in a new line.
Do not number the questions or use bullet points.
Do not mention anything before or after the list.
"""

user_message_template = """
<Document>
{document}
</Document>
"""

In [ ]:
hypothetical_questions = []

In [ ]:
documents

[Document(id='1', metadata={'year': 2023, 'section': 'business'}, page_content='We design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products.'),
 Document(id='2', metadata={'

In [ ]:
for document in documents:

    try:
        response = llm.invoke(
            [
                ('system', hypothetical_questions_system_message),
                ('user', user_message_template.format(document=document.page_content))
            ]
        )

        questions = response.content.strip()
    except Exception as e:
        questions = ""

    questions_list = questions.split("\n")

    for question in questions_list:

        questions_metadata = {
            'parent_chunk_id': document.id,
            'parent_collection': 'full_document_chunks'
        }

        hypothetical_questions.append(
            Document(
                page_content=question,
                metadata=questions_metadata
            )
        )

In [ ]:
hypothetical_questions

[Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='What types of products and services does the company provide to its customers?  '),
 Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='How does the company distribute and support its electric vehicles worldwide?  '),
 Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='What initiatives does the company pursue to lower the cost of ownership for its customers?'),
 Document(metadata={'parent_chunk_id': '2', 'parent_collection': 'full_document_chunks'}, page_content='What past supplier issues have caused delays in the launch and ramp-up of new vehicle models?  '),
 Document(metadata={'parent_chunk_id': '2', 'parent_collection': 'full_document_chunks'}, page_content='How could the introduction of new or unique manufacturing processes impact future production timelines?  '),
 Document(me

Let us look at the first set of hypothetical questions.

In [ ]:
hypothetical_questions[0], hypothetical_questions[1], hypothetical_questions[2]

(Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='What types of products and services does the company provide to its customers?  '),
 Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='How does the company distribute and support its electric vehicles worldwide?  '),
 Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='What initiatives does the company pursue to lower the cost of ownership for its customers?'))

We can now index these hypothetical questions into a new collection.

In [ ]:
hypothetical_questions_vectorstore = Chroma(
    collection_name="hypothetical_questions",
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client
)

In [ ]:
chromadb_client.list_collections()

['hypothetical_questions', 'full_document_chunks']

In [ ]:
hypothetical_questions_vectorstore.add_documents(
    documents=hypothetical_questions
)

['7d377810-b191-441b-be95-5d18a07bc5ac',
 '8cdf1b52-8b33-4ce3-92f3-2f265fc466c7',
 '9ec7cad3-8ee6-4368-bcd7-7f7ead218a87',
 '57816e21-67ec-4bef-b308-d0897d92bfc7',
 'b299a4de-aee6-4d59-9624-9f95de4c1e51',
 '6e732ac3-2844-48d1-bedc-34dd89b3ae24',
 'b5ae4f88-74ea-4b75-9e66-54698fe128c1',
 '57af8fe3-4704-42d3-817a-1a97e32837cc',
 '7553cb6d-6614-4a18-9afc-1ed534e8ca37',
 'e72b57cf-ad29-4ba6-9ec3-3a9848ede161',
 '36f862f2-35e1-4016-b797-54c61235e6b1',
 '9b3d23d8-0064-4459-85bf-b70de6e45396',
 'df50dba2-cb1f-4864-9742-9bc4f40e281b',
 'e3331d57-9f66-49c8-b06a-1f0529cc5b99',
 'e4544d00-16af-4659-8f6a-3ca7577adf5b',
 '225a0024-4221-49ea-84f7-cd5440f48c74',
 '14926210-d2b9-44ba-892c-dff0204bb7ba',
 '6ce98500-c884-490a-bd8d-183afe3d422a']

In [ ]:
retriever = hypothetical_questions_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 5}
)

In [ ]:
user_query = "Any specific fines levied on the company in 2022?"

In [ ]:
hypothetical_questions_retrieved = retriever.invoke(user_query)

In [ ]:
hypothetical_questions_retrieved

[Document(id='225a0024-4221-49ea-84f7-cd5440f48c74', metadata={'parent_chunk_id': '6', 'parent_collection': 'full_document_chunks'}, page_content='What was the final fine amount settled between Tesla and the German Umweltbundesamt?  '),
 Document(id='14926210-d2b9-44ba-892c-dff0204bb7ba', metadata={'parent_chunk_id': '6', 'parent_collection': 'full_document_chunks'}, page_content='When did the hearing take place regarding Tesla’s objection to the fine?  '),
 Document(id='6ce98500-c884-490a-bd8d-183afe3d422a', metadata={'parent_chunk_id': '6', 'parent_collection': 'full_document_chunks'}, page_content='How many times was the fine reduced and what were the amounts at each reduction?'),
 Document(id='6e732ac3-2844-48d1-bedc-34dd89b3ae24', metadata={'parent_chunk_id': '2', 'parent_collection': 'full_document_chunks'}, page_content='What risks does the company face when expanding its vehicle offerings and global footprint in terms of scaling new features?'),
 Document(id='9ec7cad3-8ee6-4368

In [ ]:
vectorstore.get(
    ids=list(set([d.metadata['parent_chunk_id'] for d in hypothetical_questions_retrieved]))
)

{'ids': ['1', '2', '6'],
 'embeddings': None,
 'documents': ['We design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products.',
  'We have previously experienced and may in the